# 2️⃣ Using Adapters from AdapterHub

In [the previous notebook](https://colab.research.google.com/github/Adapter-Hub/adapters/blob/main/notebooks/01_Adapter_Training.ipynb), we saw how to train our own adapter for a downstream task. In this notebook, we'll go through the steps to use adapters that others have trained and shared on _AdapterHub_ for **inference**.

We will use an adapter for BERT [trained on the SQuAD task](https://adapterhub.ml/explore/qa/squad1/bert/) for **extractive question answering**. This adapter achieves an F1 score of 87.75 on the dev set of SQuAD 1.1, nearly on par with full fine-tuning.

As you will see, most of the code is identical to using fully fine-tuned models with _Transformers_.

## Installation

Let's install the `adapters` libraries first:

In [ ]:
!pip install -qq adapters

## Usage

Before loading the adapter, we instantiate the model we want to use, a pre-trained `bert-base-uncased` model from HuggingFace. As before, we use _Adapters_'s [`AutoAdapterModel`](https://docs.adapterhub.ml/prediction_heads.html#adaptermodel-classes) class to be able to add a prediction head flexibly.

In [ ]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoAdapterModel.from_pretrained("bert-base-uncased")

And now to the interesting part:
Using `load_adapter()`, we can download and add a pre-trained adapter from [AdapterHub.ml](https://adapterhub.ml/explore) or from the Hugging Face Model Hub.

For this demo, we load an adapter from AdapterHub.ml. Here, the first parameter specifies the name of the adapter whereas the second selects the [adapter configuration](https://docs.adapterhub.ml/overview.html#table-of-adapter-methods) to search for. [Learn more](https://docs.adapterhub.ml/loading.html).

Also note that most adapters come with a prediction head included. Thus, this method will also load the question answering head trained together with the adapter.

In [2]:
qa_adapter = model.load_adapter("qa/squad1@ukp", config="houlsby")

With `set_active_adapters()` we tell our model to use the adapter we just loaded in every forward pass.

In [3]:
model.set_active_adapters(qa_adapter)

Now let's see our adapter in action! We create a question answering pipeline using our model and define some context text:

In [ ]:
from transformers import QuestionAnsweringPipeline

# Although the Hugging Face QuestionAnsweringPipeline does not officially support the BertAdapterModel, it does work when a qa header is enabled.
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

context = """
The current modus operandi in NLP involves downloading and fine-tuning pre-trained models consisting of millions or billions of parameters.
Storing and sharing such large trained models is expensive, slow, and time-consuming, which impedes progress towards more general and versatile NLP methods that learn from and for many tasks.
Adapters -- small learnt bottleneck layers inserted within each layer of a pre-trained model -- ameliorate this issue by avoiding full fine-tuning of the entire model.
However, sharing and integrating adapter layers is not straightforward.
We propose AdapterHub, a framework that allows dynamic "stitching-in" of pre-trained adapters for different tasks and languages.
The framework, built on top of the popular HuggingFace Transformers library, enables extremely easy and quick adaptations of state-of-the-art pre-trained models (e.g., BERT, RoBERTa, XLM-R) across tasks and languages.
Downloading, sharing, and training adapters is as seamless as possible using minimal changes to the training scripts and a specialized infrastructure.
Our framework enables scalable and easy access to sharing of task-specific models, particularly in low-resource scenarios.
AdapterHub includes all recent adapter architectures and can be found at AdapterHub.ml.
"""

In [5]:
# Ignore all FutureWarnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

Finally, we can ask our model some questions about AdapterHub:

In [6]:
def answer_questions(questions):
  for question in questions:
    result = qa(question=question, context=context)
    print("❔", question)
    print("💡", result["answer"])
    print()

answer_questions([
  "What are Adapters?",
  "What do Adapters avoid?",
  "What is proposed?",
  "What does AdapterHub allow?",
  "Where can I find AdapterHub?",
])

❔ What are Adapters?
💡 small learnt bottleneck layers inserted within each layer of a pre-trained model

❔ What do Adapters avoid?
💡 full fine-tuning of the entire model

❔ What is proposed?
💡 AdapterHub

❔ What does AdapterHub allow?
💡 dynamic "stitching-in"

❔ Where can I find AdapterHub?
💡 AdapterHub.ml



That's it! Of course, there are much more adapters available on _AdapterHub_ beyond QA adapters. Click through [our Explore page](https://adapterhub.ml/explore/) to discover all of them.

➡️ Also, the possibilities of using adapters don't stop here! Check out [the next notebook](https://colab.research.google.com/github/Adapter-Hub/adapters/blob/main/notebooks/03_Adapter_Fusion.ipynb) to see how multiple adapters can be combined for transfer learning.